In [ ]:
!git clone https://github.com/microsoft/UniPrompt.git

In [ ]:
%cd UniPrompt
!python3 -m pip install --upgrade pip setuptools
!python3 -m pip install .
%cd ..

In [9]:
config = {
    "experiment_details":"Ethos, gpt-4o, 0.7 temperature, 10 epochs, 7 batch size, 5 mini batch size, 1 iterations, 3 beam width, 2 group frequency, 0.5 epsilon",
    "dataset_path": "data/ethos.jsonl",
    "mini_batch_size": 5,
    "batch_size": 7,
    "iterations": 1,
    "epochs": 10,
    "logging_file_path": "logs/ethos.jsonl",
    "epsilon": 0.5,
    "beam_width": 3,
    "group_frequency": 2,
    "content_safety": true,
    "cache_path": "cache/ethos.db",
    "initial_prompt": "Given a query, you have to tell if it contains hate speech in any form or not.",
    "dataset_name": "ethos",
    "number_of_groups": 2, 
    "metric_kwargs": {
        "type": "accuracy"
    },
    "solver_llm": {
        "model_kwargs": {
            "temperature": 0,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    },
    "expert_llm": {
        "model_kwargs": {
            "temperature": 0.7,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    },
    "grouping_llm": {
        "model_kwargs": {
            "temperature": 0.7,
            "model": "gpt-4o"
        },
        "api_kwargs": {
            "api_type": "",
            "api_base": "",
            "api_version": "",
            "api_key": ""
        }
    }
}

In [ ]:
from uniprompt.beam_search import BeamSearch
from uniprompt.data import create_ethos_dataset, load_data
from uniprompt.evaluate import evaluate
from uniprompt.grouping import Grouping
from uniprompt.train import train
from uniprompt.utils.config_utils import load_config

In [ ]:
create_ethos_dataset("data/ethos.jsonl")
train_data, val_data, test_data = load_data(config["dataset_name"])

In [ ]:
# Initialize UniPrompt
beam = BeamSearch(config["beam_width"])
if "number_of_groups" in config:
    number_of_groups = config["number_of_groups"]
else:
    number_of_groups = 1

grouping = Grouping(number_of_groups)
p = config["initial_prompt"]

In [ ]:
metrics = evaluate(data=test_data, prompt=p, config = config)
print(f"Metrics for initial prompt: {p}: {metrics}")

In [ ]:
beam.initialize_candidates(initial_prompt = p, data=val_data, config=config)

In [ ]:
for epoch in range(config["epochs"]):
    # Create groups based on configured frequency
    p = beam.get_best_prompt()
    if epoch % config["group_frequency"] == 0:
        # if you want to group every epoch then you can do that or you can group based on the grouping frequency set in config
        # grouping function is optional, you can provide your own grouping function
        grouping.create_groups(prompt=p, data=train_data, config=config)

    # Training iterations
    for _ in range(config["iterations"]):
        print(f'Training start: {config["iterations"]}')
        beam = train(train_data=train_data, val_data=val_data, config=config, beam=beam, grouping = grouping)
        p = beam.get_best_prompt()
        # evaluation function is optional, you can provide your own evaluation function
        metrics = evaluate(data=test_data, prompt=p, config = config)
        print(f"Epoch: {epoch}, Prompt: {p}, Metrics: {metrics}")

In [ ]:
p = beam.get_best_prompt()
final_metrics = evaluate(data=test_data, prompt=p, config = config)

In [ ]:
print(f"Best prompt: {p}")
print(f"Final metrics: {final_metrics}")